<a href="https://colab.research.google.com/github/aliyaaliyal/mesin-learning/blob/main/Contoh_Transfer_Learning_dataset_Chessmen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/raw/main/ml_pengembangan_academy/Chessman-image-dataset.zip \
  -O /tmp/Chessman-image-dataset.zip

--2022-11-24 15:52:28--  https://github.com/dicodingacademy/assets/raw/main/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pengembangan_academy/Chessman-image-dataset.zip [following]
--2022-11-24 15:52:28--  https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60684125 (58M) [application/zip]
Saving to: ‘/tmp/Chessman-image-dataset.zip’

/tmp/Chessman-image 100%[===================>]  57.87M   244MB/s   

In [2]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
local_zip = '/tmp/Chessman-image-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
train_dir = os.path.join('/tmp/Chessman-image-dataset/Chess')

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode = 'nearest',
    validation_split=0.1) # set validation split

Bagi dataset menjadi data training dan data validasi.

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    subset='training') # set as training data

Found 499 images belonging to 6 classes.


In [5]:
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

Found 52 images belonging to 6 classes.


Model yang digunakan adalah ResNet152V2, dimana model ini memiliki 152 layer dan tersedia di library keras.

Untuk mengimplementasikan transfer learning, Kita hanya perlu menambahkan 2 buah baris kode berbeda. Layer pertama pada model kita adalah model yang di pakai untuk transfer learning. Kita cukup memanggil kelas ResNet152V2 dan mengisi parameter: weight, include_top dan input_tensor.

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2

In [7]:
model = tf.keras.models.Sequential([
    ResNet152V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(150, 150, 3))),
    # tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')  
])

234545216/234545216 [==============================] - 1s 0us/step


In [8]:
model.layers[0].trainable = False

selanjutnya menentukan optimizer, loss, serta metrik yang akan digunakan pada model.

In [9]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

melatih model

In [10]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              epochs=50,
                              verbose=2)

Epoch 1/50
63/63 - 94s - loss: 10.0585 - accuracy: 0.4770 - val_loss: 3.4328 - val_accuracy: 0.5385 - 94s/epoch - 1s/step
Epoch 2/50
63/63 - 83s - loss: 1.5738 - accuracy: 0.7174 - val_loss: 1.3439 - val_accuracy: 0.5577 - 83s/epoch - 1s/step
Epoch 3/50
63/63 - 84s - loss: 1.1023 - accuracy: 0.7315 - val_loss: 0.9915 - val_accuracy: 0.7308 - 84s/epoch - 1s/step
Epoch 4/50
63/63 - 84s - loss: 0.6917 - accuracy: 0.8176 - val_loss: 0.8920 - val_accuracy: 0.7308 - 84s/epoch - 1s/step
Epoch 5/50
63/63 - 84s - loss: 0.6825 - accuracy: 0.8216 - val_loss: 1.6774 - val_accuracy: 0.6346 - 84s/epoch - 1s/step
Epoch 6/50
63/63 - 84s - loss: 0.5662 - accuracy: 0.8457 - val_loss: 1.9721 - val_accuracy: 0.5962 - 84s/epoch - 1s/step
Epoch 7/50
63/63 - 83s - loss: 0.8093 - accuracy: 0.7956 - val_loss: 1.3043 - val_accuracy: 0.6538 - 83s/epoch - 1s/step
Epoch 8/50
63/63 - 84s - loss: 0.4960 - accuracy: 0.8858 - val_loss: 0.8072 - val_accuracy: 0.7885 - 84s/epoch - 1s/step
Epoch 9/50
63/63 - 83s - loss: 